In [9]:
import gensim
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [3]:
corpus = pd.read_csv("cleaned_data.csv")

In [8]:
corpus.drop("Unnamed: 0", axis = 1, inplace = True)
corpus.dropna(subset = ["text"], inplace = True)
display(corpus)

,id,title,date,text,depressive
0,train_subject1095,NaN,2014-04-30 10:52:35,the last line is pretty much the same as the ...,True
1,train_subject1095,NaN,2014-04-30 10:25:52,on the flipside generic canned ravioli is jus...,True
2,train_subject1095,NaN,2014-04-30 10:23:26,at least make it illegal to perform on infant...,True
3,train_subject1095,NaN,2014-04-30 10:20:06,i feel like i am the only who has never seen ...,True
4,train_subject1095,NaN,2014-04-30 10:16:36,if anything it was an act of god for his chut...,True
...,...,...,...,...,...
295016,train_subject9974,NaN,2015-04-02 16:52:44,yeah i have known too many privates end up wi...,False
295017,train_subject9974,NaN,2015-04-02 16:14:56,i am a soldier i am regularly away from home ...,False
295018,train_subject9974,ELI5:the button,2015-04-02 04:15:51,please explain the button at rthebutton,False
295019,train_subject9974,NaN,2015-04-02 02:33:16,stars and stripes is reporting the same thing...,False


In [10]:
ps = PorterStemmer()

In [18]:
def stemmer(text):
    words = word_tokenize(text)
    stemmed_value = ""
    for word in words:
        stemmed_value += ps.stem(word)+" "
    return stemmed_value

In [19]:
corpus ["stemmed_text"] = corpus ["text"].apply(lambda x: stemmer(x))

In [20]:
display(corpus)

,id,title,date,text,depressive,stemmed_text
0,train_subject1095,NaN,2014-04-30 10:52:35,the last line is pretty much the same as the ...,True,the last line is pretti much the same as the f...
1,train_subject1095,NaN,2014-04-30 10:25:52,on the flipside generic canned ravioli is jus...,True,on the flipsid gener can ravioli is just as go...
2,train_subject1095,NaN,2014-04-30 10:23:26,at least make it illegal to perform on infant...,True,at least make it illeg to perform on infant wh...
3,train_subject1095,NaN,2014-04-30 10:20:06,i feel like i am the only who has never seen ...,True,i feel like i am the onli who ha never seen su...
4,train_subject1095,NaN,2014-04-30 10:16:36,if anything it was an act of god for his chut...,True,if anyth it wa an act of god for hi chute not ...
...,...,...,...,...,...,...
295016,train_subject9974,NaN,2015-04-02 16:52:44,yeah i have known too many privates end up wi...,False,yeah i have known too mani privat end up with ...
295017,train_subject9974,NaN,2015-04-02 16:14:56,i am a soldier i am regularly away from home ...,False,i am a soldier i am regularli away from home f...
295018,train_subject9974,ELI5:the button,2015-04-02 04:15:51,please explain the button at rthebutton,False,pleas explain the button at rthebutton
295019,train_subject9974,NaN,2015-04-02 02:33:16,stars and stripes is reporting the same thing...,False,star and stripe is report the same thing hereh...


In [21]:
f = open("LIWC2001_English.dic")

In [30]:
data = f.read()

In [31]:
for line in data:
    print(line.split())